In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search
import json
from pandasticsearch import Select
import sys
from pathlib import Path
import datetime
import time
import pandas as pd
import config as cfg
# 일정기간 날짜 출력 방법
import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar #달력
import pandas as pd
import pymysql
import numpy as np

## 1. 엘라스틱 서치 연동 : visit data다운

In [ ]:
visit_df = pd.DataFrame()
df_g_all = pd.DataFrame()
for i in range(1,4): # 대입
    start_date = datetime.today()- timedelta(i)
    start_date2 = start_date.strftime("%Y-%m-%d")
    last_date  = datetime.today()- timedelta(i)
    last_date2  = last_date.strftime("%Y-%m-%d")
    print(start_date2, last_date2)
    # es 서버 연결
    es =  Elasticsearch("search-newspic-data-t3bcbhueisfujiwaw6lddtnbkm.ap-northeast-2.es.amazonaws.com:80", timeout=30, max_retries=10, retry_on_timeout=True)
    # doc_count 초기화
    doc_count = 0
    # 검색 요청할 인덱스 정의
    index = 'newspic-visit-'+start_date.strftime('%Y%m')
    print(index)
    # 요청쿼리문작성
    docs = es.search(index=index,  #여러개 
                      body={
                            "query": {
                              "bool": {
                                "should": [ 
                                     {"term":{'requestKey':'G9BUz12T'}}, #캐시워크 잠금화면
                                     {"term":{'requestKey':'X4uGJ61p'}} #캐시워크 drawer
                                     ],
                               "filter": [
                                    {"terms": {"requestKey": ["G9BUz12T","X4uGJ61p"]}},
                                    {"range":{
                                         "insertDate":{
                                            "gte" : start_date2,
                                            "lte" : last_date2,
                                            "time_zone": "+09:00",    
                                      }}}]
                              }
                            }
                          },
                     scroll='1m'
                    ,size=5000 )
    scroll_id = docs['_scroll_id']
    num_docs = len(docs['hits']['hits'])
    pandas_df = Select.from_dict(docs).to_pandas()
    column =['insertDate','newCategory','type','nid','pcid','requestKey','publisher','rssOption']
    pandas_df = pandas_df[column]
    a = 0 
    while num_docs > 0:
        docs = es.scroll(scroll_id=scroll_id,
                             scroll='1m')
        num_docs += len(docs['hits']['hits'])
        tdf = Select.from_dict(docs).to_pandas()
        try :
            tdf = tdf[column]
            pandas_df = pandas_df.append(tdf, ignore_index=True)
        except TypeError :
                break        
        a += 1
        print(a)
    date_column ='insertDate'
    pandas_df[date_column] = pd.to_datetime(pandas_df[date_column], unit='ms')
    pandas_df[date_column] = pandas_df[date_column].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
    pandas_df[date_column] = pandas_df[date_column].dt.tz_localize(None)
    pandas_df['insertdate']  = pandas_df[date_column].astype(str)   
    pandas_df['date']     = pandas_df['insertdate'].str.split(' ', expand = True).iloc[:,0]
    
    visit_df          = visit_df.append(pandas_df, ignore_index=True)
    
df_visit = visit_df.groupby(by=['date','rssOption','requestKey','newCategory'], as_index=False).agg({'pcid': pd.Series.nunique,
                                                                                            'nid': ['count']})
df_visit.columns =['date','rssOption','requestKey','newCategory','user','visit']
visit_df.groupby(by=['date','requestKey'], as_index=False).agg({'pcid': pd.Series.nunique,'nid': ['count']})

2022-09-04 2022-09-04
newspic-visit-202209
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
2

In [ ]:
# 데이터 타입 확인 
visit_df.info()

In [ ]:
visit_df.head()

In [ ]:
visit_df['h']= visit_df['insertdate'].str.split(' ', expand = True).iloc[:,1]
visit_df['h']= visit_df['h'].str.split(':', expand = True).iloc[:,0]

In [ ]:
visit_df['h']= visit_df['h'].str.split(':', expand = True).iloc[:,0]

In [ ]:
df_h = visit_df.groupby(by=['date','h','requestKey','rssOption'], as_index=False).agg({'pcid': pd.Series.nunique,
                                                                                            'nid': ['count']})

In [ ]:
df_h 

In [ ]:
df_h.columns=['date','h','requestKey','rssOption','user','visit']

In [ ]:
df_visit = visit_df.groupby(by=['date','rssOption','requestKey','newCategory'], as_index=False).agg({'pcid': pd.Series.nunique,
                                                                                            'nid': ['count']})

In [ ]:
df_visit.head()

In [ ]:
df_visit.to_csv("개편카테고리볕통계.csv", encoding='utf-8-sig')

In [ ]:
visit_df.groupby(by=['date','requestKey','newCategory'], as_index=False).agg({'pcid': pd.Series.nunique,
                                                                                            'nid': ['count']})

In [ ]:
df_visit.columns =['date','rssOption','requestKey','newCategory','user','visit']

In [ ]:
visit_df.groupby(by=['date','requestKey'], as_index=False).agg({'pcid': pd.Series.nunique,
                                                                                            'nid': ['count']})

## 2. 엘라스틱 서치 연동 : RSS 데이터 다운 (제공수)

In [ ]:
rss_df = pd.DataFrame()
df_g_all = pd.DataFrame()
for i in range(1,4): #i=1값 대입
    start_date = datetime.today()- timedelta(i)
    start_date2 = start_date.strftime("%Y-%m-%d")
    last_date  = datetime.today()- timedelta(i)
    last_date2  = last_date.strftime("%Y-%m-%d")
    print(start_date2, last_date2)
    # es 서버 연결
    es =  Elasticsearch("search-newspic-data-t3bcbhueisfujiwaw6lddtnbkm.ap-northeast-2.es.amazonaws.com:80", timeout=30, max_retries=10, retry_on_timeout=True)
    # doc_count 초기화
    doc_count = 0
    # 검색 요청할 인덱스 정의
    index = 'newspic-rss-'+start_date.strftime('%Y%m')
    print(index)
    # 요청쿼리문작성
    docs = es.search(index=index,  #여러개 
                      body={
                            "query": {
                              "bool": {
                            #    "should": [ 
                             #       {"term":{'requestKey':'G9BUz12T'}}, #캐시워크 잠금화면
                             #        {"term":{'requestKey':'X4uGJ61p'}} #캐시워크 drawer
                              #       ],
                                "filter": [
                                     {"terms": {"requestKey": ["G9BUz12T","X4uGJ61p"]}},
                                     {"range":{
                                         "insertDate":{
                                            "gte" : start_date2,
                                            "lte" : last_date2,
                                            "time_zone": "+09:00",    
                                      }}}]
                              }
                            }
                          },
                     scroll='1m'
                    ,size=5000)
    scroll_id = docs['_scroll_id']
    num_docs = len(docs['hits']['hits'])
    pandas_df = Select.from_dict(docs).to_pandas()
    column =['insertDate','newCategory','type','nid','requestKey','publisher','rssOption']
    pandas_df = pandas_df[column]
    a = 0 
    while num_docs > 0:
        docs = es.scroll(scroll_id=scroll_id,
                             scroll='1m')
        num_docs += len(docs['hits']['hits'])
        tdf = Select.from_dict(docs).to_pandas()
        try :
            tdf = tdf[column]
            pandas_df = pandas_df.append(tdf, ignore_index=True)
        except TypeError :
                break        
        a += 1
        print(a)
    date_column ='insertDate'
    pandas_df[date_column] = pd.to_datetime(pandas_df[date_column], unit='ms')
    pandas_df[date_column] = pandas_df[date_column].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
    pandas_df[date_column] = pandas_df[date_column].dt.tz_localize(None)
    pandas_df['insertdate']  = pandas_df[date_column].astype(str)   
    pandas_df['date']     = pandas_df['insertdate'].str.split(' ', expand = True).iloc[:,0]
    pandas_df['pcode']    = pandas_df['newCategory'].str[:4]
    rss_df          = rss_df.append(pandas_df, ignore_index=True)
df_rss    = rss_df.groupby(['date','rssOption','requestKey','newCategory'], as_index = False)['nid'].count()
df_rss.info()

In [ ]:
rss_df.head()

In [ ]:
rss_df['h']= rss_df['insertdate'].str.split(' ', expand = True).iloc[:,1]
rss_df['h']= rss_df['h'].str.split(':', expand = True).iloc[:,0]

In [ ]:
rss_h = rss_df.groupby(by=['date','h','requestKey','rssOption'], as_index=False)['nid'].count()

In [ ]:
rss_h

In [ ]:
merge_h = pd.merge(df_h,rss_h, on=['date','h','requestKey','rssOption'])

In [ ]:
merge_p = pd.pivot_table(merge_h , index=['date','h','requestKey'], values=['user','visit','nid'], aggfunc=np.sum, margins=True)
merge_p = merge_p.reset_index(drop=False, inplace=False)
merge_p.head()

In [ ]:
merge_p.columns =['date','h','requestKey','s_nid','s_user','s_visit']

In [ ]:
merge_f =  pd.merge(merge_h, merge_p, on=['date','h','requestKey'])
merge_f

In [ ]:
merge_f['ratio']= merge_f['visit']/merge_f['s_visit']*(1-(merge_f['nid']/merge_f['s_nid']))

In [ ]:
merge_f

In [ ]:
merge_f.to_csv('시간당알고리즘노출비율_220330.csv', encoding='utf-8-sig')

In [ ]:
rss_df[(rss_df['rssOption']=='RECOM_K_RTI')&(rss_df['pcode']=='CA04')]

In [ ]:
df_rss[(df_rss['rssOption']=='RECOM_K_RTI')]

In [ ]:
df_rss[(df_rss['requestKey']=='X4uGJ61p')&(df_rss['rssOption']=='RECOM_RTI')]

In [ ]:
df_rss.columns =['date','rssOption','requestKey','newCategory','nid']

In [ ]:
all_df = pd.merge(df_rss, df_visit, on=['date','rssOption','requestKey','newCategory'],how='outer')

In [ ]:
all_df.head()

In [ ]:
all_df['requestKey'].unique()

In [ ]:
all_df[all_df['requestKey']=='NONE']

## 3. DB접속 : 카테고리 테이블 매칭

In [ ]:
# 기사 제목,nid 가져오기
connect = pymysql.connect(host='ec2-3-35-194-202.ap-northeast-2.compute.amazonaws.com'
                        , port = 3306
                        , user='newspic_read'
                        , password ='shxlvmffjtm1!'
                        , db = 'newspic'
                        , charset='utf8')
# connection으로부터 cursor 생성
cur = connect.cursor()
# 카테고리 테이블 가져오기
query3 = "SELECT * FROM newsCate"
df3 = pd.read_sql(query3, connect)
df3

In [ ]:
df_all = pd.merge(all_df, df3, left_on=['newCategory'], right_on=['code'])
df_all.head(1)
df_all.info()

## 4. 퍼블리셔별 데이터 분석 : 캐시워크 잠금화면

In [ ]:
rock01  = df_all[(df_all['requestKey']=='G9BUz12T')] 

In [ ]:
###pandas pivot table sum by index
## index 1 rssoption마다 nid,user,visit의 합계를 내고 그거에 rssoption마다 비중도 말고싶다!

In [ ]:
rock01_gr= pd.pivot_table(rock01 , index=['date','rssOption','pcode'], values=['user','visit','nid'], aggfunc=np.sum, margins=True)
rock01_gr

In [ ]:
rock01_gr = rock01_gr.reset_index('pcode')
rock01_gr.head()

In [ ]:
rock01_gr_sum =rock01_gr.sum(level=['date','rssOption'])
rock01_gr_sum.columns =['nid_sum','user_sum','visit_sum']
rock01_gr_sum

In [ ]:
gr_all = pd.merge(rock01_gr, rock01_gr_sum, on=['date','rssOption'],how='left')

In [ ]:
gr_all.head()

In [ ]:
gr_all['visit_%'] = gr_all['visit']/gr_all['visit_sum']*100
gr_all['user_%'] =  gr_all['user']/gr_all['user_sum']*100
gr_all['nid_%']  =  gr_all['nid']/gr_all['nid_sum']*100
gr_all['visit/user'] = gr_all['visit']/gr_all['user']
gr_all['visit/nid']  = gr_all['visit']/gr_all['nid']

In [ ]:
gr_all=gr_all[['pcode','visit','visit_%','user','user_%',
               'nid','nid_%','visit/user','visit/nid']]

In [ ]:
gr_all = round(gr_all,2)

In [ ]:
index={ 'RECOM_RTI' : '1_실시간이슈'
       ,'RECOM_K_RTI' : '2_실시간이슈(열독률)'
       ,'RECOM_FCC' : '3_언론사&카테고리'
       ,'RECOM_CRANK' :'4_카테고리별인기'
       ,'RECOM_CTR' : '5_CTR'
       ,'RECOM_EMOJI' : '6_댓글감정표현'
       ,'RECOM_RANK': '7_인기순'
       ,'RECOM_NMEDIA' : '8_뉴미디어'
       ,'RECOM_DATE' : '9_일자별'
      }
columns ={ 'visit' :'방문수'
          ,'vist_%' :'방문수_%'
          ,'user' : '사용자수'
          ,'user_%' : '사용자수_%'
          ,'nid' : '제공기사수'
          ,'nid_%' : '제공기사수_%'
          ,'visit/user' : '제공기사당 방문수'
          ,'visit/nid' : '사용자수당 방문수'
         }
gr_all.rename(columns= columns, index=index, inplace= True)

In [ ]:
gr_all.rename(columns= columns, index=index, inplace= True)

In [ ]:
gr_all

In [ ]:
gr_all.to_csv("rock_pivot_220220_개편.csv", encoding='utf-8-sig')

## 5. 퍼블리셔별 데이터 분석 : 캐시워크 drawer

In [ ]:
drawer01  = df_all[df_all['requestKey']=='X4uGJ61p'] 

In [ ]:
###pandas pivot table sum by index
## index 1 rssoption마다 nid,user,visit의 합계를 내고 그거에 rssoption마다 비중도 말고싶다!

In [ ]:
drawer01_gr= pd.pivot_table(drawer01, index=['date','rssOption','pcode'], values=['user','visit','nid'], aggfunc=np.sum, margins=True)

In [ ]:
drawer01_gr = drawer01_gr.reset_index('pcode')

In [ ]:
drawer01_gr_sum =drawer01_gr.sum(level=['date','rssOption'])
drawer01_gr_sum.columns =['nid_sum','user_sum','visit_sum']

In [ ]:
gr_all2 = pd.merge(drawer01_gr, drawer01_gr_sum, on=['date','rssOption'],how='left')

In [ ]:
gr_all2['visit_%'] = gr_all2['visit']/gr_all2['visit_sum']*100
gr_all2['user_%'] =  gr_all2['user']/gr_all2['user_sum']*100
gr_all2['nid_%']  =  gr_all2['nid']/gr_all2['nid_sum']*100
gr_all2['visit/user'] = gr_all2['visit']/gr_all2['user']
gr_all2['visit/nid']  = gr_all2['visit']/gr_all2['nid']

In [ ]:
drawer_f=gr_all2[['pcode','visit','visit_%','user','user_%',
               'nid','nid_%','visit/user','visit/nid']]

In [ ]:
drawer_f = round(drawer_f,2)

In [ ]:
index={ 'RECOM_RTI' : '1_실시간이슈'
       ,'RECOM_K_RTI' : '2_실시간이슈(열독률)'
       ,'RECOM_FCC' : '3_언론사&카테고리'
       ,'RECOM_CRANK' :'4_카테고리별인기'
       ,'RECOM_CTR' : '5_CTR'
       ,'RECOM_EMOJI' : '6_댓글감정표현'
       ,'RECOM_RANK': '7_인기순'
       ,'RECOM_NMEDIA' : '8_뉴미디어'
       ,'RECOM_DATE' : '9_일자별'
      }
columns ={ 'visit' :'방문수'
          ,'vist_%' :'방문수_%'
          ,'user' : '사용자수'
          ,'user_%' : '사용자수_%'
          ,'nid' : '제공기사수'
          ,'nid_%' : '제공기사수_%'
          ,'visit/user' : '제공기사당 방문수'
          ,'visit/nid' : '사용자수당 방문수'
         }
drawer_f.rename(columns= columns, index=index, inplace= True)

In [ ]:
drawer_f.to_csv("drawer_pivot_220220_개편.csv", encoding='utf-8-sig')

In [ ]:
drawer_f

### 퍼블리셔별 카테고리별 방문수 비중

In [ ]:
df_all

In [ ]:
df_all_=df_all[df_all['rssOption']!='NONE']
pd.pivot_table(df_all_, index=['requestKey','date'], columns='pcode', values=['visit','nid'], aggfunc=np.sum, margins=True)

In [ ]:
df_all2=df_all[(df_all['rssOption']!='NONE')&(df_all['pcode']=='CA03')]
pd.pivot_table(df_all2, index=['requestKey','date'], columns='codeName',values=['visit','nid'], aggfunc=np.sum, margins=True)

In [ ]:
df_all2=df_all[(df_all['rssOption']!='NONE')&(df_all['pcode']=='CA01')]
pd.pivot_table(df_all2, index=['requestKey','date'], columns='codeName',values=['visit','nid'], aggfunc=np.sum, margins=True)

## 퍼블리셔별, 로직별 방문자수가 많은 기사리스트 추출

In [ ]:
nid_visit = visit_df.groupby(by=['date','rssOption','requestKey','newCategory','nid'], as_index=False).agg({'pcid': 'count'})

In [ ]:
# 기사 제목,nid 가져오기
connect = pymysql.connect(host='ec2-3-35-194-202.ap-northeast-2.compute.amazonaws.com'
                        , port = 3306
                        , user='newspic_read'
                        , password ='shxlvmffjtm1!'
                        , db = 'newspic'
                        , charset='utf8')
# connection으로부터 cursor 생성
cur = connect.cursor()
# 수집 뉴스 테이블 picNews 가져오기
query_news = '''
SELECT pn.author, pn.category, tc.category as newcategory, nc.codeName,nc.pcode, TW.*
FROM  picNews pn inner join tbContentsInfo tc on pn.nid= tc.nid
                 left join newsCate nc        ON tc.category = nc.code   
                 left JOIN tbNewsTitleWord TW ON pn.nid = TW.nid   
WHERE 1=1 
AND   pn.nid >= '2022061500000099999' 
AND   pn.nid <= '2022091523595999999'  
ORDER BY pn.pubDate DESC, pn.nid DESC
            '''
df_news = pd.read_sql(query_news, connect)
df_news = df_news[['nid','title','pcode','codeName','author','word','text_rank_word']]

In [ ]:
nid_visit.sort_values(by='pcid', ascending=False).head()
nid_visit.info()

In [ ]:
nid_visit.sort_values(by='pcid', ascending=False).head()

In [ ]:
df_news.info()

In [ ]:
nid_visit.info()

In [ ]:
df_news2 = pd.merge(nid_visit, df_news, on='nid')
df_news2.head(50)

In [ ]:
# 캐시워크 잠금화면에서 로직 인기순으로 뉴스에 나간 기사 방문수 기준으로 top5
nid_news =df_news2[['date','rssOption','requestKey','pcode','codeName','author','title','nid','pcid']]
nid_news.columns =['date','rssOption','requestKey','pcode','codeName','author','title','nid','visit']

In [ ]:
nid_news.sort_values(by='visit',ascending = False).head()

In [ ]:
nid_news.to_csv("개편카테고리별인기기사리스트_220217.csv", encoding='utf-8-sig')

## 퍼블리셔별 추천타입별 카테고리별 기사 리스트화 

In [ ]:
GROUPCA01 = (nid_news['date']=='2022-08-22')&(nid_news['requestKey']=='G9BUz12T') & (nid_news['rssOption']=='RECOM_K_RTI')&(nid_news['pcode']=='CA03')
nid_news[GROUPCA01].sort_values(by='visit',ascending = False).head()

In [ ]:
GROUPCA01 = (nid_news['date']=='2022-08-22')&(nid_news['requestKey']=='X4uGJ61p') & (nid_news['rssOption']=='RECOM_CRANK')&(nid_news['pcode']=='CA04')
nid_news[GROUPCA01].sort_values(by='visit',ascending = False).head()

## 기사별 방문수 TOP5 기사리스트

In [ ]:
# 퍼블리셔, 기사별 방문수를 집계
nid_visit2 = visit_df.groupby(by=['date','requestKey','newCategory','nid'], as_index=False).agg({'pcid': 'count'})

In [ ]:
# 기사별 방문수 테이블과 기사 테이블을 매칭화 : 기사 제목 확인
nid_news2 = pd.merge(nid_visit2, df_news, on='nid')
nid_news2.head()

In [ ]:
# 방문수 기준으로 정렬
nid_news2.sort_values(by='pcid', ascending=False)
nid_t =nid_news2[['date','requestKey','pcode','codeName', 'author','title','nid','pcid']]
nid_t.columns =['date','requestKey','pcode','codeName','author','title','nid','visit']

In [ ]:
nid_t.sort_values(by='visit', ascending=False).head()

In [ ]:
# 일자별 특정 퍼블리셔 중분류 카테고리별 기사 top5확인 
nid_t[(nid_t['date']=='2022-08-22')&(nid_t['requestKey']=='G9BUz12T')&(nid_t['codeName']=='사건사고')].sort_values(by='visit',ascending = False).head()

In [ ]:
# 일자별 특정 퍼블리셔 중분류 카테고리별 기사 top5확인
nid_t[(nid_t['date']=='2022-02-17')&(nid_t['requestKey']=='X4uGJ61p')&(nid_t['codeName']=='시건사고')].sort_values(by='visit',ascending = False).head()

In [ ]:
# 일자별 특정 퍼블리셔 중분류 카테고리별 기사 top5확인
nid_t[(nid_t['date']=='2022-06-14')&(nid_t['requestKey']=='G9BUz12T')&(nid_t['pcode']=='CA01')].sort_values(by='visit',ascending = False).head()

In [ ]:
nid_t[(nid_t['date']=='2022-06-07')&(nid_t['requestKey']=='X4uGJ61p')&(nid_t['pcode']=='CA04')].sort_values(by='visit',ascending = False).head()

In [ ]:
nid_news2

## 키워드 분석

In [ ]:
top500= nid_news2[(nid_news2['date']=='2022-09-01')&(nid_news2['requestKey']=='G9BUz12T')].sort_values(by='pcid',ascending = False).head(500)

In [ ]:
top500 = top500[['title','nid','word','text_rank_word','author','pcode','codeName','pcid']]
top500.columns = ['title','nid','word','text_rank_word','author','pcode','codeName','visit']

In [ ]:
wordlist = top500['word'].str.split(" ",expand=True)
wordlist2 = wordlist.stack().reset_index(level=1, drop=True).to_frame('keyword')

In [ ]:
df_top500 = top500.merge(wordlist2, left_index=True, right_index=True, how='left')

In [ ]:
df_top500= df_top500[['title','nid','author','codeName','visit','keyword']]

In [ ]:
pivot500 = pd.pivot_table(df_top500 , index='keyword',values='visit',aggfunc=np.sum).sort_values(by='visit', ascending=False)
dict500 = pivot500.to_dict().get('visit')
pivot500= pivot500.reset_index()
pivot500.head(20)

In [ ]:
news_df = pd.DataFrame()
for i in range(20):
    list = pivot500['keyword'].head(20)
    list3 = df_top500[df_top500['keyword'].isin([list[i]])].sort_values(['keyword','visit'], ascending=False).drop_duplicates('nid').head(3)
    news_df = news_df.append(list3)    
news_df

In [ ]:
from wordcloud import WordCloud
from PIL import Image 
import matplotlib.pylab as plt #matplotlib파이썬에서 그래프를 그릴수 있는 라이브러리

cloud_image = "C:/Users/b/python/ttf/star.png"
icon = Image.open(cloud_image).convert("RGBA")
mask = Image.new("RGB", icon.size, (255,255,255))
mask.paste(icon,icon)
mask = np.array(mask)

wordcloud = WordCloud(
            font_path ="C:/Users/b/python/ttf/BMDOHYEON.ttf" 
            ,width = 500,height =500
            ,background_color ='white'
            ,colormap = 'magma' #ocean,magma
            ,random_state = 10
            ,mask = mask
            )

In [ ]:
del(dict500['결혼'])
del(dict500['일원'])
del(dict500['이혼'])
del(dict500['이별'])
del(dict500['이유'])
del(dict500['공표'])
del(dict500['사진'])
del(dict500['연하'])
del(dict500['근황'])
del(dict500['여자친구'])
del(dict500['품절남'])
del(dict500['유부남'])
del(dict500['일반'])

In [ ]:
keyword =wordcloud.generate_from_frequencies(dict500)
array = keyword.to_array()

plt.figure(figsize=(7,7)) # figure 사이즈 변경
plt.imshow(array, interpolation="bilinear") # imshow :이미지출력
plt.axis("off") # 축표시 생략
plt.show() #부가정보 출력 생략













































## 